In [ ]:
import pytz
import datetime
import itertools
import numpy as np
import pandas as pd
from ruamel.yaml import YAML
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10, Category20
from bokeh.resources import INLINE
import panel as pn

pn.extension()

In [ ]:
ar = pytz.timezone('America/Argentina/Buenos_Aires')
now = datetime.datetime.now(ar)
dt_string = now.strftime("%d-%m-%Y")

In [ ]:
header = pn.pane.Markdown(f"""
# COVID-19 -- Argentina

- Se actualiza automáticamente a las 22:00 hora argentina. **Última actualización:** {dt_string}.
- Datos proveídos por [Sistemas Mapache](https://github.com/SistemasMapache/Covid19arData).
- El código [se distribuye](https://github.com/epassaro/covid-19) bajo la [GNU GPLv3 License](https://raw.githubusercontent.com/epassaro/covid-19/master/LICENSE).
""", width=800, margin=(0, 0, 10, 0)
)

In [ ]:
fname = 'https://docs.google.com/spreadsheets/d/16-bnsDdmmgtSxdWbVMboIHo5FRuz76DBxsz_BbsEVWA/export?format=csv&id=16-bnsDdmmgtSxdWbVMboIHo5FRuz76DBxsz_BbsEVWA&gid=0'

In [ ]:
yaml = YAML()

with open('plots.yml') as f:
    countries = yaml.load(f)

In [ ]:
from functools import reduce

def create_table(fname, countries, key):

    df = pd.read_csv(fname, index_col='fecha', parse_dates=True, dayfirst=True)
    df = df[['osm_admin_level_4', 'nue_casosconf_diff', 'nue_fallecidos_diff']]
    df.index.name = 'date'
    df.columns = ['country', 'diff', 'xdiff']
    df = df.reset_index()
    
    df_list = []
    for country in countries[key]:
        d = df.set_index('country').loc[country].reset_index(drop=True)
        d.columns = ['date', f'diff{country}', f'xdiff{country}']
        d[country] = d[f'diff{country}'].cumsum()
        d[f'x{country}'] = d[f'xdiff{country}'].cumsum()
        d[f'roll{country}'] = d[f'diff{country}'].rolling(7).mean()
        df_list.append(d)
        
    df_merged = reduce(lambda left, right: pd.merge(left, right, 
                                                    on='date', how='outer'), df_list).fillna(0)
    
    df = df_merged.groupby('date').sum()
    df = df.reset_index()
    df['datestr'] = df['date'].map(lambda x: x.strftime('%d-%m-%Y'))
    df = df.set_index('date')

    return df

In [ ]:
def make_plots(df, name):

    n = len(df.columns) // 4

    if n <= 10:
        palette = itertools.cycle(Category10[n])

    else:
        palette = itertools.cycle(Category20[n])

    source = ColumnDataSource(df)
    p = figure(plot_width=680, plot_height=420, name='Casos totales', x_axis_type='datetime')
    q = figure(plot_width=680, plot_height=420, name='Casos diarios', x_axis_type='datetime')
    r = figure(plot_width=680, plot_height=420, name='Fallecidos diarios', x_axis_type='datetime')
    s = figure(plot_width=680, plot_height=420, name='Fallecidos totales', x_axis_type='datetime')

    hovers = []
    xhovers = []
    for country in df[ [c for c in df.columns if c[:4] not in ['date', 'diff', 'roll'] \
                                                        and not c.startswith('x') ] ]:

        color = next(palette)
        p.line(x='date', y=country, source=source,
               line_width=1.5, 
               legend_label=country, 
               color=color)

        p.circle(x='date', y=country, source=source,
                 legend_label=country,
                 color=color,
                 name=country) # Tooltip are shown only on points

        q.vbar(x='date', top=f'diff{country}', source=source,
                 legend_label=country,
                 width=datetime.timedelta(days=1),
                 line_width=0,
                 color=color,
                 alpha=0.5,
                 name=country)

        q.line(x='date', y=f'roll{country}', source=source,
                 legend_label=country,
                 color=color,
                 alpha=0.75,
                 name=country)

        r.vbar(x='date', top=f'xdiff{country}', source=source,
                 legend_label=country,
                 width=datetime.timedelta(days=1),
                 line_width=0,
                 color=color,
                 alpha=0.5,
                 name=country)
        
        s.line(x='date', y=f'x{country}', source=source,
               line_width=1.5, 
               legend_label=country,
               color=color)

        s.circle(x='date', y=f'x{country}', source=source,
                 legend_label=country,
                 color=color,
                 name=country) # Tooltip are shown only on points

        TOOLTIPS = [('Provincia', country),
                    ('Fecha', '@{datestr}'),
                    ('Nuevos casos', f'@{{diff{country}}}{{0,0}}'),
                    ('Media móvil 7-d', f'@{{roll{country}}}{{0,0.0}}'),
                    ('Total casos', f'@{{{country}}}{{0,0}}')] # Double curly braces for names containing whitespaces
        
        xTOOLTIPS = [('Provincia', country),
                    ('Fecha', '@{datestr}'),
                    ('Nuevos fallecidos', f'@{{xdiff{country}}}{{0,0}}'),
                    ('Total fallecidos', f'@{{x{country}}}{{0,0}}')] # Double curly braces for names containing whitespaces

        hover = HoverTool(tooltips=TOOLTIPS, names=[country])
        hovers.append(hover)

        xhover = HoverTool(tooltips=xTOOLTIPS, names=[country])
        xhovers.append(xhover)

    p.add_tools(*hovers)
    p.legend.click_policy = 'hide'
    p.legend.location = 'top_left'
    p.legend.label_text_font_size = '8pt'
    p.yaxis.formatter = NumeralTickFormatter(format='0,0')
    p.xaxis.axis_label = "Fecha"
    p.yaxis.axis_label = "Casos totales"
    p.title.text = f"Total acumulado de casos confirmados de COVID-19 ({name})"

    q.add_tools(*hovers)
    q.legend.click_policy = 'hide'
    q.legend.location = 'top_left'
    q.legend.label_text_font_size = '8pt'
    q.yaxis.formatter = NumeralTickFormatter(format='0,0')
    q.xaxis.axis_label = "Fecha"
    q.yaxis.axis_label = "Casos diarios"
    q.title.text = f"Nuevos casos confirmados de COVID-19 por día ({name})"
    
    r.add_tools(*xhovers)
    r.legend.click_policy = 'hide'
    r.legend.location = 'top_left'
    r.legend.label_text_font_size = '8pt'
    r.yaxis.formatter = NumeralTickFormatter(format='0,0')
    r.xaxis.axis_label = "Fecha"
    r.yaxis.axis_label = "Fallecidos diarios"
    r.title.text = f"Nuevos fallecidos por COVID-19 por día ({name})"
    
    s.add_tools(*xhovers)
    s.legend.click_policy = 'hide'
    s.legend.location = 'top_left'
    s.legend.label_text_font_size = '8pt'
    s.yaxis.formatter = NumeralTickFormatter(format='0,0')
    s.xaxis.axis_label = "Fecha"
    s.yaxis.axis_label = "Fallecidos totales"
    s.title.text = f"Total acumulado de fallecidos por COVID-19 ({name})"

    return pn.Tabs(q, p, r, s, name=name, margin=(10, 20, 0, 20))

In [ ]:
_ = [ make_plots(create_table(fname, countries, k), k) for k in countries ]

In [ ]:
tabs = pn.Tabs(*_)

In [ ]:
pn.Column(header, 
          tabs, 
          margin=(20, 0, 40, 40)
         ).save('index.html', 
                title='COVID-19 - Argentina', 
                resources=INLINE)